# Data Pre-processing

In [ ]:
#library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix
import time

In [ ]:
data = pd.read_csv("winequality-white.csv", sep=";",  skiprows=[0]) # first row format is a little messed up, read data in without first row
data.loc[-1] = data.columns.to_list()
data = data.sort_index().reset_index(drop=True)
features = ["fixed acidity","volatile acidity","citric acid","residual sugar",
              "chlorides","free sulfur dioxide","total sulfur dioxide",
              "density","pH","sulphates","alcohol","quality"]
data.columns = features #add back in feature names that was in first row
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8,6
1,6.3,0.3,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6
2,8.1,0.28,0.4,6.9,0.05,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.4,9.9,6


In [ ]:
data = data.apply(pd.to_numeric, errors='coerce') #Convert values inside DataFrame to datatype float-64, by default, they are datatype Object
X = data.iloc[:,:-1] #Data (fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol)
y = data.iloc[:,-1] #Labels (quality)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) #Split into train test sets, with random seed 42 for reproducibility


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


Feature selection + KNN

In [ ]:
#Feature selection using ANOVA F-value between label/feature: classification task
selector = SelectKBest(f_classif, k=8) #Choose best features with F-value criterion, value of 8 was arbitrarily decided
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
selected_features = selector.get_support(indices=True)
feature_names = X.columns[selected_features]

In [ ]:
data[feature_names] # after feature selection:

,fixed acidity,volatile acidity,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,alcohol
0,7.0,0.27,20.7,0.045,45.0,170.0,1.00100,8.8
1,6.3,0.30,1.6,0.049,14.0,132.0,0.99400,9.5
2,8.1,0.28,6.9,0.050,30.0,97.0,0.99510,10.1
3,7.2,0.23,8.5,0.058,47.0,186.0,0.99560,9.9
4,7.2,0.23,8.5,0.058,47.0,186.0,0.99560,9.9
...,...,...,...,...,...,...,...,...
4893,6.2,0.21,1.6,0.039,24.0,92.0,0.99114,11.2
4894,6.6,0.32,8.0,0.047,57.0,168.0,0.99490,9.6
4895,6.5,0.24,1.2,0.041,30.0,111.0,0.99254,9.4
4896,5.5,0.29,1.1,0.022,20.0,110.0,0.98869,12.8


In [ ]:
scaler = MinMaxScaler() #normalizes data for KNN classifier, inproves accuracy of results slightly
X_train_selected = scaler.fit_transform(X_train_selected)
X_test_selected = scaler.transform(X_test_selected)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
start = time.time() # start timing the training process
knn.fit(X_train_selected, y_train)
end = time.time() # end timing the training process

print("Training time:", end - start, "seconds")

Training time: 0.005276918411254883 seconds


In [ ]:
y_pred = knn.predict(X_test_selected)
accuracy = knn.score(X_test_selected, y_test) # Evaluate the classifier on the test set
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.53


In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm)
df_cm.columns = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8'] #column names represnet what was predicted
df_cm.index = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8'] #index names represents ground truth
df_cm

,Quality 3,Quality 4,Quality 5,Quality 6,Quality 7,Quality 8
Quality 3,0,0,4,3,0,0
Quality 4,0,2,20,10,0,0
Quality 5,0,4,214,128,12,0
Quality 6,0,1,142,340,59,2
Quality 7,0,0,13,132,90,6
Quality 8,0,0,2,23,14,4


Feature selection + LDA

In [ ]:
# reload data, reinitialize variables
data = pd.read_csv("winequality-white.csv", sep=";",  skiprows=[0])
data.loc[-1] = data.columns.to_list()
data = data.sort_index().reset_index(drop=True)
features = ["fixed acidity","volatile acidity","citric acid","residual sugar",
              "chlorides","free sulfur dioxide","total sulfur dioxide",
              "density","pH","sulphates","alcohol","quality"]
data.columns = features
data = data.apply(pd.to_numeric, errors='coerce') #Convert values inside DataFrame to datatype float-64, by default, they are datatype Object
X = data.iloc[:,:-1] #Data (fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol)
y = data.iloc[:,-1] #Labels (quality)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42) #Split into train test sets, with random seed 42 for reproducibility


In [ ]:
#Feature selection using ANOVA F-value between label/feature: classification task
selector = SelectKBest(f_classif, k=8) #Choose best features with F-value criterion
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
selected_features = selector.get_support(indices=True)
feature_names = X.columns[selected_features]


In [ ]:
scaler = MinMaxScaler() #normalizes data for KNN classifier, inproves accuracy of results slightly
X_train_selected = scaler.fit_transform(X_train_selected)
X_test_selected = scaler.transform(X_test_selected)

In [ ]:
lda = LDA()
start = time.time() # start timing the training process
lda.fit(X_train_selected, y_train)
end = time.time() # end timing the training process
print("Training time:", end - start, "seconds")

Training time: 0.008979320526123047 seconds


In [ ]:
y_pred = lda.predict(X_test_selected)
accuracy = lda.score(X_test_selected, y_test) # Evaluate the classifier on the test set
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51


In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm)
df_cm.columns = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8']
df_cm.index = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8']
df_cm

,Quality 3,Quality 4,Quality 5,Quality 6,Quality 7,Quality 8
Quality 3,1,0,3,3,0,0
Quality 4,0,8,9,15,0,0
Quality 5,0,11,174,170,2,1
Quality 6,1,6,107,387,43,0
Quality 7,0,0,17,166,58,0
Quality 8,0,1,0,26,16,0


PCA + KNN

In [ ]:
# reload data, reinitialize variables
data = pd.read_csv("winequality-white.csv", sep=";",  skiprows=[0])
data.loc[-1] = data.columns.to_list()
data = data.sort_index().reset_index(drop=True)
features = ["fixed acidity","volatile acidity","citric acid","residual sugar",
              "chlorides","free sulfur dioxide","total sulfur dioxide",
              "density","pH","sulphates","alcohol","quality"]
data.columns = features
data = data.apply(pd.to_numeric, errors='coerce') #Convert values inside DataFrame to datatype float-64, by default, they are datatype Object
X = data.iloc[:,:-1] #Data (fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol)
y = data.iloc[:,-1] #Labels (quality)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#PCA feature selection
pca = PCA(n_components=8)
X_train_selected = pca.fit_transform(X_train)
X_test_selected = pca.transform(X_test)

In [ ]:
scaler = MinMaxScaler() #normalizes data for KNN classifier, inproves accuracy of results slightly
X_train_selected = scaler.fit_transform(X_train_selected)
X_test_selected = scaler.transform(X_test_selected)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)
start = time.time() # start timing the training process
knn.fit(X_train_selected, y_train)
end = time.time() # end timing the training process
print("Training time:", end - start, "seconds")

Training time: 0.007528781890869141 seconds


In [ ]:
y_pred = knn.predict(X_test_selected)
accuracy = knn.score(X_test_selected, y_test) # Evaluate the classifier on the test set
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.55


In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm)
df_cm.columns = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8']
df_cm.index = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8']
df_cm

,Quality 3,Quality 4,Quality 5,Quality 6,Quality 7,Quality 8
Quality 3,1,0,2,4,0,0
Quality 4,0,3,19,10,0,0
Quality 5,0,3,229,115,10,1
Quality 6,0,3,135,350,55,1
Quality 7,0,0,20,131,83,7
Quality 8,0,0,3,22,16,2


PCA + LDA

In [ ]:
# reload data, reinitialize variables
data = pd.read_csv("winequality-white.csv", sep=";",  skiprows=[0])
data.loc[-1] = data.columns.to_list()
data = data.sort_index().reset_index(drop=True)
features = ["fixed acidity","volatile acidity","citric acid","residual sugar",
              "chlorides","free sulfur dioxide","total sulfur dioxide",
              "density","pH","sulphates","alcohol","quality"]
data.columns = features
data = data.apply(pd.to_numeric, errors='coerce') #Convert values inside DataFrame to datatype float-64, by default, they are datatype Object
X = data.iloc[:,:-1] #Data (fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol)
y = data.iloc[:,-1] #Labels (quality)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
#PCA feature selection
pca = PCA(n_components=8)
X_train_selected = pca.fit_transform(X_train)
X_test_selected = pca.transform(X_test)

In [ ]:
scaler = MinMaxScaler() #normalizes data for KNN classifier, inproves accuracy of results slightly
X_train_selected = scaler.fit_transform(X_train_selected)
X_test_selected = scaler.transform(X_test_selected)

In [ ]:
lda = LDA()
start = time.time() # start timing the training process
lda.fit(X_train_selected, y_train)
end = time.time() # end timing the training process
print("Training time:", end - start, "seconds")

Training time: 0.01381993293762207 seconds


In [ ]:
y_pred = lda.predict(X_test_selected)
accuracy = lda.score(X_test_selected, y_test) # Evaluate the classifier on the test set
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.51


In [ ]:
cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm)
df_cm.columns = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8']
df_cm.index = ['Quality 3', 'Quality 4', 'Quality 5', 'Quality 6', 'Quality 7', 'Quality 8']
df_cm

,Quality 3,Quality 4,Quality 5,Quality 6,Quality 7,Quality 8
Quality 3,1,0,4,2,0,0
Quality 4,0,0,14,18,0,0
Quality 5,0,0,187,168,3,0
Quality 6,2,0,124,375,43,0
Quality 7,0,0,27,154,60,0
Quality 8,0,0,2,21,20,0
